<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame

import random  

import keras
import tensorflow as tf
from keras import backend as K
from keras.layers import LSTM, TimeDistributed, BatchNormalization, Bidirectional, Dense, Dropout
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers.recurrent import GRU
from keras.layers import *
from keras.utils.vis_utils import plot_model

import time

Using TensorFlow backend.


In [62]:
UNITS = 32  # Units 数目
EPOCHS = 2  # 训练循环数目
TEST_SIZE = 0.2  # 数据集分割比例
VAL = 0.2  # val 分割比例
BATCH_SIZE = 32  # 一次训练多少组
seed = 1340  # 随机数种子
PATIENCE = 20  # 允许不降落步数
OUTPUT_SIZE = 1  # 输入一个 输出 1个
LR = 0.006            # 误差期望
filepath1 = 'train_3.xlsx'
filepath2 = "Dose_inter_slinear.xlsx"

In [58]:
def data_pre(filepath1, filepath2):
    df = pd.read_excel(filepath1)  # 读取文件
    training_set = DataFrame(df)
    training_set = training_set.T

    ct = pd.read_excel(filepath2, header=None)
    ct = np.array(ct)
    ct = ct[:, :250]

    X_train_0 = training_set.iloc[:, :302].values
    y_train_0 = training_set.iloc[:, 302:].values
    X_train_0 = X_train_0[:, :250]
    y_train_0 = y_train_0[:, :250]  # 修建 0 值

    X_train_total = np.zeros((200, 250, 2))
    for i in range(200):
        X_train_total[i, :, 0] = X_train_0[i, :]
    for i in range(200):
        X_train_total[i, :, 1] = ct[i % 10, :]

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        X_train_total, y_train_0, test_size=TEST_SIZE, random_state=seed)

    #X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))
    #X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))

    return X_train, y_train, X_test, y_test, X_train_total, y_train_0

In [96]:
model = Sequential()
model.add(
    LSTM(input_shape=(250, 2),
        units=UNITS,
        return_sequences=True,
        name="GRU"))

#model.add(Dropout(0.10, name="dropout"))
model.add(LSTM(units=UNITS, return_sequences=True, name="GRU_2"))
model.add(LSTM(units=UNITS, return_sequences=True, name="GRU_3"))
model.add(
    TimeDistributed(Dense(OUTPUT_SIZE, name="Dense"), name="TDense"))
adam = Adam(LR)

model.compile(
    optimizer=adam,
    loss='mse',
)

In [83]:
model = Sequential()
model.add(
    Bidirectional(
        GRU(
            input_shape=(250, 2),
            units=UNITS,
            return_sequences=True,
            name="GRU_1"),
        name="BiGRU_1"))

#model.add(Dropout(0.10, name="dropout_2"))

model.add(
    Bidirectional(
        GRU(
            input_shape=(250, 2),
            units=UNITS,
            return_sequences=True,
            name="GRU_2"),
        name="BiGRU_2"))
model.add(
    Bidirectional(
        GRU(
            input_shape=(250, 2),
            units=UNITS,
            return_sequences=True,
            name="GRU_3"),
        name="BiGRU_3"))
model.add(TimeDistributed(Dense(OUTPUT_SIZE, name="Dense"), name="TDense"))
adam = Adam(LR)

model.compile(
    optimizer=adam,
    loss='mse',
)

In [97]:
X_train, y_train, X_test, y_test, X_train_0, y_train_0 = data_pre(filepath1,filepath2)
history = model.fit(
    X_train,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=2,
    verbose=1,
    validation_split=0.2,  # 加入 交叉验证集
    )

Train on 128 samples, validate on 32 samples
Epoch 1/2
128/128 [==============================] - 6s 46ms/step - loss: 0.0785 - val_loss: 0.0508
Epoch 2/2
128/128 [==============================] - 2s 15ms/step - loss: 0.0470 - val_loss: 0.0441


In [98]:
model.load_weights('LSTM_DLoss.h5',by_name= True)

In [86]:
def loss_cal(X_test, y_test):

    # 计算 LOSS 峰值偏移等指标

    N = X_test.shape[0]

    test_loss_total = []
    peak_loss_total = []
    peak_shift_total = []
    peak_half_loss_list = []
    peak_twenty_percent_loss_list = []

    for i in range(N):

        X_test[i] = np.array(X_test[i], dtype=float)
        y_test[i] = np.array(y_test[i], dtype=float)
        X_test_0 = X_test[i][np.newaxis, :, :]
        y_test_0 = y_test[i][np.newaxis, :, :]

        # 计算 测试集总误差

        y_test_0_pred = model.predict(X_test_0)
        test_loss = np.mean(np.square(y_test_0_pred - y_test_0))
        test_loss_total.append(test_loss)

        x_pre = model.predict(X_test_0)
        x_pre = x_pre.ravel()  # 调整形状
        x_pre = np.array(x_pre, dtype=np.float64)  # 调整 数据
        x_real = y_test_0.ravel()

        # 计算测试集 peak 偏移
        peak_shift = abs(np.argmax(x_real) - np.argmax(x_pre))
        peak_shift_total.append(peak_shift)

        # 转换为列表
        x_real_list = x_real.tolist()
        x_pre_list = x_pre.tolist()

        # 计算 测试集 距离峰值一半的偏差
        half_peak = x_real.max() * 0.5
        for j in x_real[:np.argmax(x_real)]:
            if j > half_peak or j == half_peak:
                index_l1 = x_real_list.index(j)
                break
        for j in x_real[np.argmax(x_real):]:
            if j < half_peak or j == half_peak:
                index_r1 = x_real_list[np.argmax(x_real):].index(
                    j) + np.argmax(x_real)
                break

        half_loss = [
            abs(x_real_list[i] - x_pre_list[i])
            for i in range(index_l1,index_r1)
        ]
        peak_half_loss = np.mean(np.square(half_loss))
        peak_half_loss_list.append(peak_half_loss)

        # 计算距离峰值 20% 的偏差
        twenty_percent_peak = x_real.max() * 0.2
        for i in x_real[:np.argmax(x_real)]:
            if i > twenty_percent_peak or i == twenty_percent_peak:
                index_l2 = x_real_list.index(i)
                break

        for i in x_real[np.argmax(x_real):]:
            if i < twenty_percent_peak or i == twenty_percent_peak:
                index_r2 = x_real_list[np.argmax(x_real):].index(
                    i) + np.argmax(x_real)
                break
        twenty_percent_loss = [
            abs(x_real_list[i] - x_pre_list[i])
            for i in range(index_l2,index_r2)
        ]
        peak_twenty_percent_loss = np.mean(np.square(twenty_percent_loss))
        peak_twenty_percent_loss_list.append(peak_twenty_percent_loss)

        i = i + 1

    peak_shift_MAE = np.mean(peak_shift_total)
    peak_shift_MSE = np.mean(np.square(peak_shift_total))
    peak_twenty_percent_loss_ave = np.mean(peak_twenty_percent_loss_list)
    peak_half_loss_ave = np.mean(peak_half_loss_list)
    test_loss_total_ave = np.mean(test_loss_total)

    return peak_shift_MAE, peak_shift_MSE,test_loss_total_ave, peak_half_loss_ave,peak_twenty_percent_loss_ave

In [87]:
def curve_shift(X_test, y_test):

    per_average_total = []
    per50_average_total = []
    per20_average_total = []
    N = X_test.shape[0]

    for i in range(N):

        X_test[i] = np.array(X_test[i], dtype=float)
        y_test[i] = np.array(y_test[i], dtype=float)
        X_test_0 = X_test[i][np.newaxis, :, :]
        y_test_0 = y_test[i][np.newaxis, :, :]
        #X_test[i] = X_test[i].reshape(X_test[i].shape(1),X_test[i].shape(0),1)
        #y_test[i] = y_test[i].reshape(y_test[i].shape(1),y_test[i].shape(0),1)

        

        x_pre = model.predict(X_test_0, batch_size=1, verbose=0)
        x_pre = x_pre.ravel()  # 调整形状
        x_pre = np.array(x_pre, dtype=np.float64)  # 调整 数据
        x_real = y_test_0.ravel()

        # 计算曲线的 总偏移
        #curve_shift = x_real - x_pre
        per_list = []
        n = 0
        for i in range(240):
            if x_real[i] >  1e-3:
                n= n+1
                a= abs(x_real[i] - x_pre[i] )/ x_real[i] 
                per_list.append(a)
        per_average = sum(per_list) / n
        per_average_total.append(per_average)

        # 计算曲线的半高偏移

        # 转换为列表
        x_real_list = x_real.tolist()
        x_pre_list = x_pre.tolist()

        # 计算 测试集 距离峰值一半的偏差
        half_peak = x_real.max() * 0.5
        for j in x_real[:np.argmax(x_real)]:
            if j > half_peak or j == half_peak:
                index_l1 = x_real_list.index(j)
                break
        for j in x_real[np.argmax(x_real):]:
            if j < half_peak or j == half_peak:
                index_r1 = x_real_list[np.argmax(x_real):].index(
                    j) + np.argmax(x_real)
                break

        per50_list = [
            abs(x_real_list[i] - x_pre_list[i]) / x_real_list[i]
            for i in range(index_l1,index_r1)
        ]

        per50_average = sum(per50_list) / (index_r1 - index_l1)
        per50_average_total.append(per50_average)

        # 计算距离峰值 20% 的偏差
        twenty_percent_peak = x_real.max() * 0.2
        for i in x_real[:np.argmax(x_real)]:
            if i > twenty_percent_peak or i == twenty_percent_peak:
                index_l2 = x_real_list.index(i)
                break

        for i in x_real[np.argmax(x_real):]:
            if i < twenty_percent_peak or i == twenty_percent_peak:
                index_r2 = x_real_list[np.argmax(x_real):].index(
                    i) + np.argmax(x_real)
                break
        per20_list = [
            abs(x_real_list[i] - x_pre_list[i]) / x_real_list[i]
            for i in range(index_l2,index_r2)
        ]

        per20_average = sum(per20_list) / (index_r2 - index_l2)
        per20_average_total.append(per20_average)

        i = i + 1

    per100 = sum(per_average_total) / N
    per50 = sum(per50_average_total) / N
    per20 = sum(per20_average_total) / N

    return per100, per50, per20

In [74]:
loss_cal(X_test,y_test)

(0.55,
 0.65,
 0.0005031892497434117,
 0.002509077264587034,
 0.0004634449879963386)

In [38]:
curve_shift(X_test,y_test)

(0.23501883399527218, 0.051138979541799724, 0.055561868605934726)

In [23]:
def activity_add_noise(X_train_0,Inputsize, Inputlength, SNR):
    x_train_noise = X_train_0*(np.ones((Inputsize, Inputlength)) + np.random.randn(Inputsize, Inputlength)/SNR)
    return x_train_noise

In [40]:
X_test_SNR = activity_add_noise(
    X_test[:,:,0].reshape(X_test.shape[0], X_test.shape[1]), X_test.shape[0],
    X_test.shape[1], 9)

In [41]:
X_train_total = np.zeros((40,250,2))
for i in range(40):
    X_train_total[i,:,0] = X_test_SNR[i,:]
for i in range(40):
    X_train_total[i,:,1] = X_test[:,:,1][i,:]

In [43]:
loss_cal(X_train_total,y_test)

(0.5, 0.55, 0.0007814954149639123, 0.0046508680941777306, 0.001360477639338479)

In [44]:
curve_shift(X_train_total,y_test)

(0.25482290909677435, 0.07312883025344576, 0.07019523652850454)

In [99]:
peak_shift_MAE_list = []
test_loss_total_ave_list = []
peak_half_loss_ave_list = []
peak_twenty_percent_loss_ave_list = []

per100_list = []
per50_list = []
per20_list = []

for i in range(9):
    X_test_SNR = activity_add_noise(
        X_test[:,:,0].reshape(X_test.shape[0], X_test.shape[1]), X_test.shape[0],
        X_test.shape[1], i + 1)
    X_train_total = np.zeros((40, 250, 2))
    for i in range(40):
        X_train_total[i, :, 0] = X_test_SNR[i, :]
    for i in range(40):
        X_train_total[i, :, 1] = X_test[:, :, 1][i, :]
    peak_shift_MAE, peak_shift_MSE, test_loss_total_ave, peak_half_loss_ave, peak_twenty_percent_loss_ave = loss_cal(
        X_train_total, y_test)
    per100, per50, per20 = curve_shift(X_train_total, y_test)

    peak_shift_MAE_list.append(peak_shift_MAE)
    test_loss_total_ave_list.append(test_loss_total_ave)
    peak_half_loss_ave_list.append(peak_half_loss_ave)
    peak_twenty_percent_loss_ave_list.append(peak_twenty_percent_loss_ave)
    per100_list.append(per100)
    per50_list.append(per50)
    per20_list.append(per20)

In [100]:
per50_list

[0.3831591776444699,
 0.271376443783086,
 0.1718220426538223,
 0.13304751989921537,
 0.10463109070582861,
 0.09681636405121485,
 0.0866977241830744,
 0.07923593628406388,
 0.07899709789992411]

In [101]:
GRU_loss_save = pd.DataFrame({
    'peak_shift_MAE_list':
    peak_shift_MAE_list,
    'test_loss_total_ave_list':
    test_loss_total_ave_list,
    'peak_half_loss_ave_list':
    peak_half_loss_ave_list,
    'peak_twenty_percent_loss_ave_list':
    peak_twenty_percent_loss_ave_list,
    'per100_list':
    per100_list,
    'per50_list':
    per50_list,
    'per20_list':
    per20_list
})
GRU_loss_save.to_csv("LSTM_loss.csv", index=False, sep=',')